### Transformation

Now we need to go into linear algebra and matrices. If you have not done any matrix math before, I will try to
keep this simple. In linear algebra we have 1-dimensional arrays, like a point. We have written the point class
with X, Y, Z member variables. But, it could have been written as an array of 3 values with X, Y, and Z indices of
0, 1, and 2 respectively.

This prepares us to talk about a transform. The idea is that there is a transformation matrix that maps from one
coordinate system (i.e. robot coordinates) to another coordinate system (i.e. field coordinates). The transformation
is a 4x4 matrix, and the transformation (`xfm`) from coordinate system to another coordinate system written as:
```
     [ xfm[0][0] xfm[0][1] xfm[0][2] xfm[0][3] ] [ X ]   [ Xt ]
     | xfm[1][0] xfm[1][1] xfm[1][2] xfm[1][3] | | Y | = | Yt |
     | xfm[2][0] xfm[2][1] xfm[2][2] xfm[2][3] | | Z |   | Zt |
     [ xfm[3][0] xfm[3][1] xfm[3][2] xfm[3][3] ] [ 1 ]   [ 1  ]
```

If you have not done matrix math before, the multiplication is that you walk horizontally across each row of the
matrix and multiply the column value by the corresponding row value of the point, then sum the result.
 Confused? - OK lets walk through that:
```
    Xt = (xfm[0][0] * X) + (xfm[0][1] * Y) + (xfm[0][2] * Z) + (xfm[0][3] * 1)
    Yt = (xfm[1][0] * X) + (xfm[1][1] * Y) + (xfm[1][2] * Z) + (xfm[1][3] * 1)
    Zt = (xfm[2][0] * X) + (xfm[2][1] * Y) + (xfm[2][2] * Z) + (xfm[2][3] * 1)
    1  = (xfm[3][0] * X) + (xfm[3][1] * Y) + (xfm[3][2] * Z) + (xfm[3][3] * 1)
```

And why does that all work out? Well, the heading is a rotation around the Z axis (NOTE, however, as described in
the coordinate system part of the previous section, we are using right-hand coordinate systems for the field and
the robot - and in a right-hand coordinate system a positive Z rotation rotates X into Y, or is counter-clockwise;
we have been using heading as a clockwise rotation, so we need to negate it for the vector geometry
formalization); and the position of the
robot on the field, call it `(Tx,Ty,Tz=0)` (for translation in X and Y) results in a transformation of:

```
     [  cos(-heading) sin(-heading) 0.0   Tx ] [  X  ]   [  Xt ]
     | -sin(-heading) cos(-heading) 0.0   Ty | |  Y  | = |  Yt |
     |      0.0           0.0       1.0   Tz | |  Z  |   |  Zt |
     [      0.0           0.0       0.0  1.0 ] [ 1.0 ]   [ 1.0 ]
```

And when we walk through the transformation multipltation:
```
    Xt = ((cos(-heading) * X)  + (sin(-heading) * Y) + 0.0 +  Tx)
    Yt = ((-sin(-heading) * X) + (cos(-heading) * Y) + 0.0 +  Ty)
    Zt = (       0.0           +       0.0           +  Z  +  Tz)
    1  = (       0.0           +       0.0           + 0.0 + 1.0)
```


In the last section we wrote a bit of code to see what 3d points, lines, and vectors looked like in code. We
aren't going to write 3d transformation code - or use our previous, example code. Instead, I've added all of
that to our `a05annexUtil` library and we will be using that so the code we write here looks like the
code we would write for the robot. The next cell is the instructions to connect to that library. 

***FIRST*** *fun fact*: the robot is a new effort every season. Specifically, you are supposed to build a new robot and write new programming every season with one
specific exception: designs or code that are publically posted so they are available to other teams may be used
in the next seasons. This is why all of our code it on github (open source), and we have packaged and
released our `a05annexUtil` library so it can be automatically loaded from the open source maven repositories.

The `a05annexUtil` with the 3d vector geometry code is currently in a *staging* repository. This is a test
repository so we can use and test it here or on our robot before we officially release it to the Maven open
source repositories used by the *gradle* build tools. The `<repository>`
section of the next cell describes where the repository is on the internet, and the `<dependency>` section
describes our `a05annexUtil` in that repository:

In [72]:
%%loadFromPOM
<repository>
  <id>oss-sonatype-staging</id>
  <url>https://oss.sonatype.org/content/repositories/staging/</url>
</repository>

<dependency>
  <groupId>org.a05annex</groupId>
  <artifactId>a05annexUtil</artifactId>
  <version>0.8.9</version>
</dependency>

When you run the cell above you should not get any output, if you do get output, that means there is a problem. For
example, if you change the version to `0.8.10` and run the cell there will be an error `[unresolved dependency: org.a05annex#a05annexUtil;0.8.10: not found]` because there is no version `0.8.10`. **NOTE:** once we are done testing and release this library it wil be in the *releases* area rather than *staging*.

OK, let's do a little test to make sure we can get to the `a05annexUtil` 3d vector math classes:

In [70]:
import org.a05annex.util.geo3d.Point3d;
import org.a05annex.util.geo3d.Vector3d;

// create a couple points
Point3d pt0 = new Point3d(0.0, 0.0, 0.0);
Point3d pt1 = new Point3d(1.0, 2.0, 3.0);
// get the direction vector from pt0 to pt1
Vector3d v = new Vector3d(pt0,pt1).normalize();
// and print out some info to verify
System.out.println("pt0: (" + pt0.getX() + "," + pt0.getY() + "," + pt0.getZ() + ")");
System.out.println("pt1: (" + pt1.getX() + "," + pt1.getY() + "," + pt1.getZ() + ")");
System.out.println("v: (" + v.getI() + "," + v.getJ() + "," + v.getK() + ")");
System.out.println("Length v is: " + v.getLength());


pt0: (0.0,0.0,0.0)
pt1: (1.0,2.0,3.0)
v: (0.2672612341463735,0.534522468292747,0.8017837024391204)
Length v is: 1.0


Excellent, we are using the new library and we are ready to do some really cool stuff.

### Field to/from Robot Transformations

Whoa - that last section was a lot to digest, so let's look a transformation as it relates to us.
This section will describe transformation from robot coordinates to field coordinates, and from
field coordinates to robot coordinates. In our 2020-2021 swerve drive code we added code to
track where we think the robot is on the field
based on our control commands - this does not account for slippage on the field or collisions with
other robots. In our path planning we transformed the robot coordinates (the geometry of the robot)
to field coordinates to display robot position on the field and to test for the robot extending past
field boundaries. In our exploration of trying to use field elements to determine where the robot
is on the field we determine where the field elements are relative to the robot and work backwards
from that to figure out where where the robot is on the field.

Let's look at the
infinite recharge field; specifically the high port in the scoring goal with the large hexagonal primary
front face goal that has the limelight tape, and the smaller circular goal that is centered on the
hexagon; 1'-1" in diameter and 2'-5&frac14; (0.74m)" behind the primary front face. So, let us diagram that
remembering that our convention for path planning is that the origin of the field coordinate
system, `(0.0, 0.0, 0.0)`, is the field center. We have a several standard start positions (though these
are normally the back bumper of the robot) midfield at
the start line `(0.0, 5.0, 0.0)`, right (in front of goal) at the start line `(1.7085, 5.0, 0.0)`, and left
(in front of the ball pickup) at `(-1.56, 5.0, 0.0)`. For shooting from the end of the trench to protect us from defense, the location is `(2.695, 2.743, 0.0)`:

![alt text](./resources/InfiniteRechargeField.jpg "infinite recharge field")


Where do we go from here? Lets start by building a cell that defines the key points of the playing field:

In [71]:
static final int FIELD_CENTER = 0;
static final int START_CENTER = 1;
static final int START_RIGHT = 2;
static final int START_LEFT = 3;
static final int TRENCH_CORNER = 4;
static final int HEX_GOAL = 5;
static final int CENTER_GOAL = 6;
static final String[] FIELD_NAMES = {"field center     : ",
                                     "start center     : ",
                                     "start right      : ",
                                     "start left       : ",
                                     "trench corner    : ",
                                     "hex goal center  : ",
                                     "round goal center: "};
static final Point3d[] FIELD_POINTS = {new Point3d(0.0, 0.0, 0.0),
                                       new Point3d(0.0, 5.0, 0.0),
                                       new Point3d(1.7085, 5.0, 0.0),
                                       new Point3d(-1.56, 5.0, 0.0),
                                       new Point3d(2.695, 2.743, 0.0),
                                       new Point3d(1.7085, 7.99, 2.5),
                                       new Point3d(1.7085, 8.73, 2.5)};

public void printFieldPts(Point3d[] fieldPts)
{
    for (int i = 0; i < fieldPts.length; i++) {
        System.out.println(String.format("%s (%9.3f,%9.3f,%9.3f)", FIELD_NAMES[i],
                           fieldPts[i].getX(), fieldPts[i].getY(), fieldPts[i].getZ()));
    }
}

System.out.println("-- field locations --");
printFieldPts(FIELD_POINTS);


-- field locations --
field center     :  (    0.000,    0.000,    0.000)
start center     :  (    0.000,    5.000,    0.000)
start right      :  (    1.709,    5.000,    0.000)
start left       :  (   -1.560,    5.000,    0.000)
trench corner    :  (    2.695,    2.743,    0.000)
hex goal center  :  (    1.709,    7.990,    2.500)
round goal center:  (    1.709,    8.730,    2.500)


And lets create a class that represents our robot, we will use that to keep track of robot geometry, camera geometry on the robot, and robot position on the field. For the robot geometry, we will be a really skinny robot so our orientation is really obvious in the transformations:

In [60]:
import org.a05annex.util.AngleD;
import org.a05annex.util.geo3d.Point3d;
import org.a05annex.util.geo3d.Xfm4x4d;

class MyRobot {
    
    // The robot coordinates
    static final int CENTER = 0;
    static final int RF = 1;
    static final int LF = 2;
    static final int LR = 3;
    static final int RR = 4;
    static final int LIMELIGHT = 5;

    /* The Z position (relative to the robot) of the limelight lens in meters. */
    static final double LIMELIGHT_HEIGHT = .52;
    /* The elevation angle of the camera relative to the XY plane (the floor) in radians. */
    static final double LIMELIGHT_ELEV_ANGLE = 0.337;
    /* The X position (relative to the robot) of the limelight lens in meters. */
    static final double LIMELIGHT_X_OFFSET = -0.20;
    /* The Y position (relative to the robot) of the limelight lens in meters. */
    static final double LIMELIGHT_Y_OFFSET = 0.14;
    
    static final String[] GEOMETRY_NAMES = {"center     : ", "right front: ", "left front : ",
                                            "left rear  : ", "right rear : ", "limelight  : " };
    static final Point3d[] ROBOT_GEOMETRY = {new Point3d(0.0, 0.0, 0.0), new Point3d(0.1, 0.3, 0.0),
        new Point3d(-0.1, 0.3, 0.0), new Point3d(-0.1, -0.3, 0.0), new Point3d(0.1, -0.3, 0.0),
        new Point3d(LIMELIGHT_X_OFFSET, LIMELIGHT_Y_OFFSET, LIMELIGHT_HEIGHT)};
    
    double m_fieldX = 0.0;
    double m_fieldY = 0.0;
    final AngleD m_heading = new AngleD(AngleD.RADIANS, 0.0);
    
    final Xfm4x4d m_robotToFieldXfm = new Xfm4x4d();
    final Xfm4x4d m_fieldToRobotXfm = new Xfm4x4d();
    
    public MyRobot()
    {
    }
    
    public MyRobot(double fieldX, double fieldY, AngleD heading)
    {
        setFieldPosition(fieldX, fieldY, heading);
    }
    
    /**
     * Set the position of the robot on the field.
     */
    public void setFieldPosition(double fieldX, double fieldY, AngleD heading)
    {
        m_fieldX = fieldX;
        m_fieldY = fieldY;
        m_heading.setValue(heading);
        // build the transform that positions the robot relative to the field
        // by initializing the transform to an identify, applying the heading
        // rotation, and then translating the robot to some position on the field.
        m_robotToFieldXfm.identity();
        m_robotToFieldXfm.rotate(Xfm4x4d.AXIS_Z, heading.cloneAngleD().mult(-1.0));
        m_robotToFieldXfm.translate(fieldX, fieldY, 0.0);
        // invert that to buld the transform that describes where the field
        // elements are relative to the robot
        m_robotToFieldXfm.invert(m_fieldToRobotXfm);
        
    }
    
    /**
     * Transform an array of field-relative coordinates to robot-relative
     * coordinates. This takes an array of field-relative coordinates, and
     * creates an array of robot-relative coordinates. The field-relative
     * coordinates are unchanged.
     */
    public Point3d[] xfmFieldPtsToRobot(Point3d[] fieldPts)
    {
        Point3d[] robotPts = new Point3d[fieldPts.length];
        for (int i = 0; i < fieldPts.length; i++) {
            robotPts[i] = fieldPts[i].clonePoint3d();
        }
        m_fieldToRobotXfm.transform(robotPts);
        return robotPts;
    }
    
    /**
     * Transform the robot-relative geometry to field-relative coordinates and
     * return those field-relative coordinates. This is useful if you want to
     * display the robot on a diagram of the field.
     */
    public Point3d[] xfmRobotPtsToField()
    {
        Point3d[] fieldPts = new Point3d[ROBOT_GEOMETRY.length];
        for (int i = 0; i < ROBOT_GEOMETRY.length; i++) {
            fieldPts[i] = ROBOT_GEOMETRY[i].clonePoint3d();
        }
        m_robotToFieldXfm.transform(fieldPts);
        return fieldPts;
    }
    
    /**
     * Print an annotatted list of the robot geometry transformed to field-relative
     * coordinates for the current field position. This is primarily a debugging method.
     */
    public void printRobotPtsToField()
    {
        Point3d[] pts = xfmRobotPtsToField();
        for (int i = 0; i < pts.length; i++) {
            System.out.println(String.format("%s (%9.3f,%9.3f,%9.3f)", GEOMETRY_NAMES[i],
                                             pts[i].getX(), pts[i].getY(), pts[i].getZ()));
        }
    }
}

MyRobot myRobot = new MyRobot();

System.out.println("-- Raw robot coordinates when the robot is at center field --");
myRobot.printRobotPtsToField();
System.out.println("\n-- Robot field coordinates when at center field rotated 90 degrees --");
myRobot.setFieldPosition(0.0, 0.0, new AngleD(AngleD.DEGREES,90.0));
myRobot.printRobotPtsToField();

System.out.println("\n-- Robot field coordinates when centered on the center of the start line --");
myRobot.setFieldPosition(FIELD_POINTS[START_CENTER].getX(), FIELD_POINTS[START_CENTER].getY(), new AngleD(AngleD.RADIANS,0.0));
myRobot.printRobotPtsToField();
System.out.println("\n-- Robot field coordinates when centered on the center of the start line, heading 30 degrees --");
myRobot.setFieldPosition(FIELD_POINTS[START_CENTER].getX(), FIELD_POINTS[START_CENTER].getY(), new AngleD(AngleD.DEGREES,30.0));
myRobot.printRobotPtsToField();
System.out.println("-- Field elements relative to the robot --");
printFieldPts(myRobot.xfmFieldPtsToRobot(FIELD_POINTS));
System.out.println("\n-- Robot field coordinates when centered on the center of the start line, heading 90 degrees --");
myRobot.setFieldPosition(FIELD_POINTS[START_CENTER].getX(), FIELD_POINTS[START_CENTER].getY(), new AngleD(AngleD.DEGREES,90.0));
myRobot.printRobotPtsToField();
System.out.println("-- Field elements relative to the robot --");
printFieldPts(myRobot.xfmFieldPtsToRobot(FIELD_POINTS));


-- Raw robot coordinates when the robot is at center field --
center     :  (    0.000,    0.000,    0.000)
right front:  (    0.100,    0.300,    0.000)
left front :  (   -0.100,    0.300,    0.000)
left rear  :  (   -0.100,   -0.300,    0.000)
right rear :  (    0.100,   -0.300,    0.000)
limelight  :  (   -0.200,    0.140,    0.520)

-- Robot field coordinates when at center field rotated 90 degrees --
center     :  (    0.000,    0.000,    0.000)
right front:  (    0.300,   -0.100,    0.000)
left front :  (    0.300,    0.100,    0.000)
left rear  :  (   -0.300,    0.100,    0.000)
right rear :  (   -0.300,   -0.100,    0.000)
limelight  :  (    0.140,    0.200,    0.520)

-- Robot field coordinates when centered on the center of the start line --
center     :  (    0.000,    5.000,    0.000)
right front:  (    0.100,    5.300,    0.000)
left front :  (   -0.100,    5.300,    0.000)
left rear  :  (   -0.100,    4.700,    0.000)
right rear :  (    0.100,    4.700,    0.000)
limeligh

When we review this we see that the field position after any translaion and/or rotation is what we expected with
some round-off error. Note that when the robot is centered on the start line and turns to a heading of 30&deg;
the robot is almost directly facing the shooting goal (the X offset in robot coordinates is almost 0.0).

OK, this is pretty cool, but how do we work the limelight into this? Well, lets create a `Limelight` class that
represents the limelight relative to the robot so we can build the transformations between limelight coordinates
and robot coordinates. The `Limelight` class will look almost like the `MyRobot` class except that it will be
expressing the relationship between the limelight and the robot instead of the robot and the field:

In [64]:
import org.a05annex.util.AngleD;
import org.a05annex.util.geo3d.Point3d;
import org.a05annex.util.geo3d.Xfm4x4d;

class Limelight {
    
    // The limelight coordinates
    static final int CENTER = 0;
    static final int UNIT_X = 1;
    static final int UNIT_Y = 2;
    static final int UNIT_Z = 3;

    static final String[] LIMELIGHT_NAMES = {"center : ",
                                             "unit X : ",
                                             "unit Y : ",
                                             "unit Z : "};
    static final Point3d[] LIMELIGHT_GEOMETRY = {new Point3d(0.0, 0.0, 0.0),
        new Point3d(1.0, 0.0, 0.0), new Point3d(0.0, 1.0, 0.0), new Point3d(0.0, 0.0, 1.0)};
    
    double m_robotX = 0.0;
    double m_robotY = 0.0;
    double m_robotZ = 0.0;
    final AngleD m_elevation = new AngleD(AngleD.RADIANS, 0.0);
    
    Xfm4x4d m_limelightToRobotXfm = new Xfm4x4d();
    Xfm4x4d m_robotToLimelightXfm = new Xfm4x4d();
    
    public Limelight()
    {
    }
    
    public Limelight(double robotX, double robotY, double robotZ, AngleD elevation)
    {
        setLimelightPosition(robotX, robotY, robotZ, elevation);
    }
    
    /**
     * Set the position of the limelight on the robot.
     */
    public void setLimelightPosition(double robotX, double robotY, double robotZ, AngleD elevation)
    {
        m_robotX = robotX;
        m_robotY = robotY;
        m_robotZ = robotZ;
        m_elevation.setValue(elevation);
        // build the transform that positions the limelight relative to the robot
        // by initializing the transform to an identify, applying a scaling to map
        // from a left handed coordinate system to a right handed, a rotation to
        // set the elevation angle, and a translation to set position of the limelight
        // on the robot.
        m_limelightToRobotXfm.identity();
        // rescale the axis system from a left handed to right handed axis system
        m_limelightToRobotXfm.scale(1.0, 1.0, -1.0);
        // rotate around X until the Y is straight up (90deg) plus the tilt angle for the
        // camera facing forward
        AngleD xRotation = new AngleD(AngleD.DEGREES, 90.0).add(m_elevation);
        m_limelightToRobotXfm.rotate(Xfm4x4d.AXIS_X, xRotation);
        m_limelightToRobotXfm.translate(m_robotX, m_robotY, m_robotZ);
        // invert that to buld the transform that describes where the robot
        // elements are relative to the limelight
        m_limelightToRobotXfm.invert(m_robotToLimelightXfm);
        
    }
    
    /**
     * Transform an array of robot-relative coordinates to limelight-relative
     * coordinates. This takes an array of robot-relative coordinates, and
     * creates an array of limelight-relative coordinates. The robot-relative
     * coordinates are unchanged.
     */
    public Point3d[] xfmRobotPtsToLimelight(Point3d[] robotPts)
    {
        Point3d[] limelightPts = new Point3d[robotPts.length];
        for (int i = 0; i < robotPts.length; i++) {
            limelightPts[i] = robotPts[i].clonePoint3d();
        }
        return m_robotToLimelightXfm.transform(limelightPts);
    }
    
    /**
     * Transform the limelight-relative geometry to robot-relative coordinates and
     * return those robot-relative coordinates. This is useful if you want to
     * display the limelight on a diagram of the robot.
     */
    public Point3d[] xfmLimelightPtsToRobot()
    {
        Point3d[] robotPts = new Point3d[LIMELIGHT_GEOMETRY.length];
        for (int i = 0; i < LIMELIGHT_GEOMETRY.length; i++) {
            robotPts[i] = LIMELIGHT_GEOMETRY[i].clonePoint3d();
        }
        return m_limelightToRobotXfm.transform(robotPts);
    }
    
    /**
     * Print an annotatted list of the limelight geometry transformed to robot-relative
     * coordinates for the current camera position. This is primarily a debugging method.
     */
    public void printLimelightPtsToRobot()
    {
        Point3d[] pts = xfmLimelightPtsToRobot();
        for (int i = 0; i < pts.length; i++) {
            System.out.println(String.format("%s (%7.3f,%7.3f,%7.3f)", LIMELIGHT_NAMES[i],
                                             pts[i].getX(), pts[i].getY(), pts[i].getZ()));
        }
    }
}

Limelight myLimelight = new Limelight();
myLimelight.setLimelightPosition(0.0,0.0,0.0,new AngleD(AngleD.DEGREES,0.0));
System.out.println("-- Robot coordinates of limelight at (0,0,0), elevation = 0");
myLimelight.printLimelightPtsToRobot();

myLimelight.setLimelightPosition(-0.200, 0.140, 0.520, new AngleD(AngleD.DEGREES,0.0));
System.out.println("\n-- Robot coordinates of limelight at (-0.200,0.140,0.520), elevation = 0");
myLimelight.printLimelightPtsToRobot();

myLimelight.setLimelightPosition(-0.200, 0.140, 0.520, new AngleD(AngleD.RADIANS,0.337));
System.out.println("\n-- Robot coordinates of limelight at (-0.200,0.140,0.520), elevation = 0.337rad");
myLimelight.printLimelightPtsToRobot();

System.out.println("\n-- Robot field coordinates when centered on the center of the start line, heading 30 degrees --");
myRobot.setFieldPosition(FIELD_POINTS[START_CENTER].getX(), FIELD_POINTS[START_CENTER].getY(), new AngleD(AngleD.DEGREES,30.0));
myRobot.printRobotPtsToField();
System.out.println("-- Field elements relative to the robot --");
printFieldPts(myRobot.xfmFieldPtsToRobot(FIELD_POINTS));
System.out.println("-- Field elements relative to the limelight --");
printFieldPts(myLimelight.xfmRobotPtsToLimelight(myRobot.xfmFieldPtsToRobot(FIELD_POINTS)));




-- Robot coordinates of limelight at (0,0,0), elevation = 0
center :  (  0.000,  0.000,  0.000)
unit X :  (  1.000,  0.000,  0.000)
unit Y :  (  0.000,  0.000,  1.000)
unit Z :  (  0.000,  1.000, -0.000)

-- Robot coordinates of limelight at (-0.200,0.140,0.520), elevation = 0
center :  ( -0.200,  0.140,  0.520)
unit X :  (  0.800,  0.140,  0.520)
unit Y :  ( -0.200,  0.140,  1.520)
unit Z :  ( -0.200,  1.140,  0.520)

-- Robot coordinates of limelight at (-0.200,0.140,0.520), elevation = 0.337rad
center :  ( -0.200,  0.140,  0.520)
unit X :  (  0.800,  0.140,  0.520)
unit Y :  ( -0.200, -0.191,  1.464)
unit Z :  ( -0.200,  1.084,  0.851)

-- Robot field coordinates when centered on the center of the start line, heading 30 degrees --
center     :  (    0.000,    5.000,    0.000)
right front:  (    0.237,    5.210,    0.000)
left front :  (    0.063,    5.310,    0.000)
left rear  :  (   -0.237,    4.790,    0.000)
right rear :  (   -0.063,    4.690,    0.000)
limelight  :  (   -0.103, 

And, this last bit, `-- Field elements relative to the limelight --` is really interesting
to us, especially the `hex goal center`. If we imagine the limelight tape was a square at the center
of the hex target, then the point (0.185,0.776,3.773) is the position of the centroid of the limelight
target relative to the limelight camera lens (at (0,0,0)). So the limelight X angle is
atan2(0.185,3.773) and the Y angle is atan2(0.776,3.773):

In [68]:
AngleD limelightX = new AngleD().atan2(0.185,3.773);
System.out.println(String.format("limelight X angle %.2f degree (%.3f radians)",
    limelightX.getDegrees(), limelightX.getRadians()));
    
AngleD limelightY = new AngleD().atan2(0.776,3.773);
System.out.println(String.format("limelight Y angle %.2f degree (%.3f radians)",
    limelightY.getDegrees(), limelightY.getRadians()));

limelight X angle 2.81 degree (0.049 radians)
limelight Y angle 11.62 degree (0.203 radians)


OK, now we have the basic components of the math we need to compute where the robot is on the field. We will save working through the details of that for the next session.